In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from sklearn.dummy import DummyClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

from sklearn.multioutput import MultiOutputClassifier

In [4]:
df = pd.read_csv('../Datasets/Traffic_Crashes_-_Vehicles.csv')

C:\Users\mitch\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,60,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.columns

Index(['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO',
       'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE',
       'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT',
       'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER',
       'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY',
       'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I',
       'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I',
       'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',
       'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       '

In [6]:
df.shape

(1311810, 72)

In [7]:
clean_vehicles = df[['CRASH_RECORD_ID', 'VEHICLE_ID', 'UNIT_TYPE', 'NUM_PASSENGERS', 'MAKE', 'MODEL',
                     'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'MANEUVER', 'FIRST_CONTACT_POINT']]

In [8]:
clean_vehicles.shape

(1311810, 11)

In [26]:
clean_vehicles.isna().sum()

CRASH_RECORD_ID              0
VEHICLE_ID               29628
UNIT_TYPE                 1843
NUM_PASSENGERS         1116747
MAKE                     29633
MODEL                    29777
VEHICLE_DEFECT           29628
VEHICLE_TYPE             29628
VEHICLE_USE              29628
MANEUVER                 29628
FIRST_CONTACT_POINT      40767
dtype: int64

In [27]:
# Although this looks like a lot missing (30k), it's only 2.3% of data, so I'll drop these.
clean_vehicles.dropna(subset=['VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'MANEUVER', 'MODEL', 'MAKE',
                             'UNIT_TYPE', 'FIRST_CONTACT_POINT'], inplace=True)
clean_vehicles.isna().sum()

<ipython-input-27-91cde20c3917>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles.dropna(subset=['VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'MANEUVER', 'MODEL', 'MAKE',


CRASH_RECORD_ID              0
VEHICLE_ID                   0
UNIT_TYPE                    0
NUM_PASSENGERS         1077555
MAKE                         0
MODEL                        0
VEHICLE_DEFECT               0
VEHICLE_TYPE                 0
VEHICLE_USE                  0
MANEUVER                     0
FIRST_CONTACT_POINT          0
dtype: int64

In [28]:
# We still have ~97% of the data by just dropping the nans, and that's good enough for me
clean_vehicles.shape[0]/df.shape[0]

0.9688018844192375

In [29]:
# Assume that there are 0 passengers if not specified
clean_vehicles['NUM_PASSENGERS'].fillna(0, inplace=True)

C:\Users\mitch\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [30]:
clean_vehicles.isna().sum()

CRASH_RECORD_ID        0
VEHICLE_ID             0
UNIT_TYPE              0
NUM_PASSENGERS         0
MAKE                   0
MODEL                  0
VEHICLE_DEFECT         0
VEHICLE_TYPE           0
VEHICLE_USE            0
MANEUVER               0
FIRST_CONTACT_POINT    0
dtype: int64

In [31]:
# pickle df to combine with crash and vehicle df's
clean_vehicles.to_pickle("./Data/clean_vehicles.pkl")